# Crash Map Video

Make a video showing crashes over time on a map

Data source: https://www.denvergov.org/opendata/dataset/city-and-county-of-denver-traffic-accidents

Based on: https://medium.com/udacity/creating-map-animations-with-python-97e24040f17b

In [1]:
import os
import pytz
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib._color_data import TABLEAU_COLORS

import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM, Stamen
import pandas as pd
from tqdm.notebook import tqdm
import geopy
import geopy.distance

os.chdir('..')
from scripts.crash_data_analysis import CrashDataAnalysis
cda = CrashDataAnalysis()

In [2]:
import cartopy.io.img_tiles as cimgt

class StadiaStamen(cimgt.Stamen):
#     def __init__():
    def _image_url(self, tile):
        stadia_api_key = os.environ.get('STADIA_API_KEY')
        x,y,z = tile
        r = '@2x'
        # url = f"https://tiles.stadiamaps.com/tiles/stamen_terrain/{z}/{x}/{y}.png?api_key={stadia_api_key}"
        url = f'https://tiles.stadiamaps.com/tiles/stamen_toner_lite/{z}/{x}/{y}{r}.png?api_key={stadia_api_key}'
        return url

# Retrieves tiles from maps.stamen.com. Styles include terrain-background, terrain, toner and watercolor.
imagery = StadiaStamen('toner')

Documentation: https://stadiamaps.com/stamen/onboarding/migrate

* https://tiles.stadiamaps.com/tiles/stamen_toner/{z}/{x}/{y}{r}.png?api_key=
* https://tiles.stadiamaps.com/tiles/stamen_toner_lite/{z}/{x}/{y}{r}.png?api_key=
* https://tiles.stadiamaps.com/tiles/stamen_terrain/{z}/{x}/{y}{r}.jpg?api_key=
* https://tiles.stadiamaps.com/tiles/stamen_watercolor/{z}/{x}/{y}.jpg?api_key=

Note: if you're using a library that doesn't support {r} (e.g., MapLibre), replace {r} with @2x or drop the placeholder to use standard 256x256 px tiles.

Attribution: 
&copy; <a href="https://stadiamaps.com/" target="_blank">Stadia Maps</a>
&copy; <a href="https://www.stamen.com/" target="_blank">Stamen Design</a>
&copy; <a href="https://openmaptiles.org/" target="_blank">OpenMapTiles</a>
&copy; <a href="https://www.openstreetmap.org/copyright/" target="_blank">OpenStreetMap</a>

In [3]:
# East Speer

# frame_directory = 'videos/speer'
# paste_center = [39.726591, -104.986417]
# miles_vertical = 0.9
# zoom_level = 16

In [4]:
# Westwood

# frame_directory = 'videos/kentucky'
# paste_center = [39.70039857002305, -105.03918676552246]
# miles_vertical = 1.7
# zoom_level = 15

In [5]:
# Downtown - Broadway / 20th St / California

# frame_directory = 'videos/downtown'
# paste_center = [39.74891627011233, -104.98736403434121]
# miles_vertical = 1.2
# zoom_level = 16

In [6]:
# CDOT HQ

# frame_directory = 'videos/cdot'
# paste_center = [39.738465044125576, -105.02321500866896]
# miles_vertical = 0.5
# zoom_level = 17

In [7]:
# Federal, between 20th and 50th avenues

frame_directory = 'videos/federal'
paste_center = [39.76559481781074, -105.02526809191475]
miles_vertical = 3.5
zoom_level = 15

In [8]:
mrct = cda.most_recent_crash_timestamp()

date_first = pytz.timezone('America/Denver').localize(datetime(2023, 1, 1))

date_last = pytz.timezone('America/Denver').localize(datetime(2024, 1, 1))
# date_last = mrct + timedelta(days=28)

video_date_range = pd.date_range(date_first, date_last, freq='d')

In [9]:
if not os.path.exists(frame_directory):
    os.makedirs(frame_directory)

In [10]:
d_vertical = geopy.distance.distance(miles=(miles_vertical)/2)

# wide
# d_horizontal = geopy.distance.distance(miles=(miles_vertical * 1.25)/2)

# narrow for Federal
d_horizontal = geopy.distance.distance(miles=(miles_vertical * 0.5)/2)

center = geopy.Point(latitude=paste_center[0], longitude=paste_center[1])

bounds = {}
bounds['left'] = d_horizontal.destination(point=center, bearing=270).longitude
bounds['top'] = d_vertical.destination(point=center, bearing=0).latitude
bounds['right'] = d_horizontal.destination(point=center, bearing=90).longitude
bounds['bottom'] = d_vertical.destination(point=center, bearing=180).latitude

In [11]:
d_horizontal

Distance(1.408176)

In [12]:
square_miles = (miles_vertical * 1.25) * miles_vertical
square_miles

15.3125

In [13]:
query = f"""
select 
*
, reported_date at time zone 'America/Denver' as reported_date_local

from crashes

where {bounds['left']} < geo_lon
and geo_lon < {bounds['right']}
and {bounds['bottom']} < geo_lat
and geo_lat < {bounds['top']}

and '{date_first}' <= reported_date at time zone 'America/Denver'
and reported_date at time zone 'America/Denver' <= '{date_last}'

order by reported_date
"""

df_in_bounds = pd.read_sql(query, cda.conn)

In [14]:
print('Stats about crash database: ')
print(f'Number of crashes: {len(df_in_bounds):,}')
print(f'Unique IDs: {df_in_bounds.incident_id.nunique():,}')
print(f'Min date: {df_in_bounds.reported_date_local.min()}')
print(f'Max date: {df_in_bounds.reported_date_local.max()}')

Stats about crash database: 
Number of crashes: 1,222
Unique IDs: 1,222
Min date: 2023-01-01 01:23:00
Max date: 2023-12-31 23:21:00


In [15]:
df_in_bounds['reported_date_local'] = df_in_bounds['reported_date_local'].dt.tz_localize(
    'America/Denver', ambiguous='NaT'
)

In [16]:
stats_by_date = df_in_bounds.groupby('crash_date').agg(
    crashes=('incident_id', 'size')
    , fatalities=('fatality', 'sum')
    , serious_injuries=('sbi', 'sum')
    , pedestrians=('pedestrian_ind', 'sum')
    , bicycles=('bicycle_ind', 'sum')
)

# stats_by_date.to_clipboard()

stats_by_date.sum()

crashes             1222.0
fatalities             5.0
serious_injuries      21.0
pedestrians           34.0
bicycles              13.0
dtype: float64

In [17]:
df_in_bounds['marker_shape'] = 'o' # circle
df_in_bounds.loc[df_in_bounds.pedestrian_ind > 0, 'marker_shape'] = 'D' # diamond
df_in_bounds.loc[df_in_bounds.bicycle_ind > 0, 'marker_shape'] = 's' # square

# https://colorbrewer2.org/#type=diverging&scheme=RdYlBu&n=4
# https://colorbrewer2.org/#type=qualitative&scheme=Dark2&n=3
colors = {}
colors['crashes'] = '#1b9e77' # green
colors['serious_injuries'] = '#d95f02' # orange
colors['fatalities'] = '#7570b3' # purple

df_in_bounds['marker_color'] = colors['crashes']
df_in_bounds.loc[df_in_bounds.sbi, 'marker_color'] = colors['serious_injuries']
df_in_bounds.loc[df_in_bounds.fatality, 'marker_color'] = colors['fatalities']

In [18]:
# # Save the base map
# # todo: Stamen no longer supported
# imagery = Stamen(style='toner-lite', cache=True)

In [19]:
fig = plt.figure(figsize=[16,9])
fig.subplots_adjust(left=0, bottom=0.05, right=0.7, top=0.95)

ax = plt.axes(projection=imagery.crs)
ax.add_image(imagery, zoom_level)
ax.set_extent((bounds['left'], bounds['right'], bounds['bottom'], bounds['top']))

# plt.savefig(f'{frame_directory}/base_map.png', facecolor='white')
plt.close()

In [20]:
def save_one_frame(df, map_date, bounds, zoom_level, frame_filename, colors):
    """
    Save one frame of the video with crashes from that day and previous days
    
    df : DataFrame containing crashes (already limited to the bounds). If None, only base map is saved
    map_date : date for the crashes that will be plotted
    bounds : dict containing left (lon), right (lon), top (lat), and bottom (lat) bounds of the map
    frame_filename : the filename where to save the frame PNG
    colors : dict containing colors for 'crashes', 'serious_injuries', and 'fatalities'
    """

    fig = plt.figure(figsize=[16,9])

    fig.subplots_adjust(left=0, bottom=0.05, right=0.7, top=0.95)

    ax = plt.axes(projection=imagery.crs)
    ax.set_extent((bounds['left'], bounds['right'], bounds['bottom'], bounds['top']))
    ax.add_image(imagery, zoom_level)
    
    if frame_directory == 'videos/cdot':
        plt.gcf().text(
            x=0.35
            , y=0.53
            , s='Colorado Department\nof Transportation HQ'
            , fontsize=14
            , font='Menlo'
            , color='white'
            , backgroundcolor=TABLEAU_COLORS['tab:red']
            , horizontalalignment='center'
            , verticalalignment='center'
        )
    
    
    for days_back in range(27,-1,-1):
        
        rows_to_plot = df.crash_date == (map_date - timedelta(days=days_back)).strftime('%Y-%m-%d')

        if rows_to_plot.sum() > 0:
            for m in df[rows_to_plot]['marker_shape'].unique():
                for c in df[rows_to_plot]['marker_color'].unique():
                    
                    # Make the crashes fade out sooner than the injuries and fatalities
                    if c == colors['crashes']:
                        fill_style='none'
                        if days_back >= 14:
                            alpha_value = 0
                        elif days_back >= 7:
                            alpha_value = 0.8 - (0.1 * (days_back - 6))
                        else:
                            alpha_value = 0.8

                    else: # fatalities and serious injuries
                        fill_style = 'full'
                        if days_back >= 21:
                            alpha_value = 0.8 - (0.1 * (days_back - 20))
                        else:
                            alpha_value = 0.8


                    ax.plot(
                        df.loc[
                            (rows_to_plot) & (df['marker_shape'] == m) & (df['marker_color'] == c)
                            , 'geo_lon'
                            ].to_numpy()
                        , df.loc[
                            (rows_to_plot) & (df['marker_shape'] == m) & (df['marker_color'] == c)
                            , 'geo_lat'
                            ].to_numpy()
                        , transform=ccrs.Geodetic()
                        , marker=m
                        , color=c
                        , linestyle=''
                        , markersize=(5*(days_back+1)) + 10
                        , fillstyle=fill_style
                        , markeredgewidth=6
                        , alpha=alpha_value
                    )


    # Do counts of crashes up to map_date
    counts = df[df.reported_date_local <= map_date].groupby('crash_date').agg(
        crashes=('incident_id', 'size')
        , fatalities=('fatality', 'sum')
        , serious_injuries=('sbi', 'sum')
        , pedestrians=('pedestrian_ind', 'sum')
        , bicycles=('bicycle_ind', 'sum')
    ).sum()    
    
    if map_date > mrct:
        date_for_legend = mrct.strftime('%Y-%m-%d')
    else:
        date_for_legend = map_date.strftime('%Y-%m-%d')
        
    legend = [
        ('Date: {}'.format(date_for_legend), 'black', 0.91)
        , ('Car crashes:{: 6,.0f}'.format(counts['crashes']), colors['crashes'], 0.78)
        , ('Crashes with a       \nserious injury:{: 6,.0f}'.format(
            counts['serious_injuries']), colors['serious_injuries'], 0.6175)
        , ('Crashes with a       \nfatality:{: 6,.0f}'.format(counts['fatalities']), colors['fatalities'], 0.455)
        , (u"\n\u25C7 Pedestrian       \ninvolved:{: 6,.0f}".format(counts['pedestrians']), 'black', 0.2925)
        , (u"\n\u25FB Bicycle       \ninvolved:{: 6,.0f}".format(counts['bicycles']), 'black', 0.13)
    ]
    
    for idx, line in enumerate(legend):
            
        plt.gcf().text(
            x=0.97
            , y=line[2]
            , s=line[0]
            , fontsize=26
            , horizontalalignment='right'
            , verticalalignment='bottom'
#             , family='monospace'
            , font='Menlo'
            , color=line[1]
        )

    plt.gcf().text(
        x=0.97
        , y=0.05
        , s='Visualization by Devin Brady'
        , fontsize=18
        , horizontalalignment='right'
#         , family='monospace'
        , font='Menlo'
        , color='black'
    )

    plt.savefig(frame_filename, facecolor='white')
    plt.close()
    
    return legend

In [21]:
# Save the initial map
_ = save_one_frame(
    pd.DataFrame(columns=df_in_bounds.columns)
    , video_date_range[0]
    , bounds
    , zoom_level
    , f'{frame_directory}/initial_map.png'
    , colors
)

In [22]:
for counter, d in enumerate(tqdm(video_date_range, 'Saving map frames')):

    # only save the last image
#     if counter < 364:
#         continue
    
    legend = save_one_frame(df_in_bounds, d, bounds, zoom_level, f'{frame_directory}/frame_{counter:04d}.png', colors)


Saving map frames:   0%|          | 0/366 [00:00<?, ?it/s]

## Terminal command 

In [23]:
print(
    f'ffmpeg -framerate 12 -i {frame_directory}/frame_%4d.png -c:v h264 -r 30 '
    + f'-s 1920x1080 {frame_directory}/crashes.mp4'
)

ffmpeg -framerate 12 -i videos/federal/frame_%4d.png -c:v h264 -r 30 -s 1920x1080 videos/federal/crashes.mp4


In [24]:
# import numpy as np
# np.linspace(0.75, 0.1, num=5)

In [25]:
# import matplotlib
# matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf') #[:10]

In [26]:
for leg in legend:
    print(leg[0])

Date: 2024-01-01
Car crashes: 1,221
Crashes with a       
serious injury:    21
Crashes with a       
fatality:     5

◇ Pedestrian       
involved:    34

◻ Bicycle       
involved:    13


In [27]:
df_in_bounds[df_in_bounds.fatality]

,incident_id,top_traffic_accident_offense,reported_date,incident_address_corrected,at_freeway,geo_lon,geo_lat,neighborhood_id,bicycle_ind,pedestrian_ind,...,crash_date,crash_date_str,crash_time_str,crash_year,crash_day_of_year,geom,geom_denver,reported_date_local,marker_shape,marker_color
132,202378997,TRAF - ACCIDENT - FATAL,2023-02-13 11:30:00+00:00,INTERSTATE 70 / N LOWELL BLVD,True,-105.034576,39.783449,Berkeley,0.0,1.0,...,2023-02-13,2023-02-13 Mon,"Mon Feb 13, 4:30 AM",2023,44,0101000020E6100000701F5E7C36425AC0C9AB730C48E4...,0101000020AE0D000062291600DBE24741E7630D00FB19...,2023-02-13 04:30:00-07:00,D,#7570b3
551,2023316260,TRAF - ACCIDENT - FATAL,2023-06-15 07:22:00+00:00,2900 BLOCK W COLFAX AVE,False,-105.024140,39.740342,Sun Valley,0.0,1.0,...,2023-06-15,2023-06-15 Thu,"Thu Jun 15, 1:22 AM",2023,166,0101000020E6100000AD2D02848B415AC0B43EC48AC3DE...,0101000020AE0D00005AADF97FBEE84741D534A3FFB3DC...,2023-06-15 01:22:00-06:00,D,#7570b3
701,2023419512,TRAF - ACCIDENT - FATAL,2023-08-05 05:22:00+00:00,W COLFAX AVE / N NEWTON ST,False,-105.036985,39.740344,West Colfax,0.0,0.0,...,2023-08-04,2023-08-04 Fri,"Fri Aug 4, 11:22 PM",2023,216,0101000020E6100000EEF651F45D425AC0D7231E9AC3DE...,0101000020AE0D00008AA60480B0E14741AB311D00A2DC...,2023-08-04 23:22:00-06:00,o,#7570b3
862,2023495795,TRAF - ACCIDENT - FATAL,2023-09-14 07:17:00+00:00,W 44TH AVE / N FEDERAL BLVD,False,-105.025225,39.776642,Berkeley,0.0,0.0,...,2023-09-14,2023-09-14 Thu,"Thu Sep 14, 1:17 AM",2023,257,0101000020E6100000621079479D415AC0DC1AC7FE68E3...,0101000020AE0D00000B9F148003E84741E207B4FF5810...,2023-09-14 01:17:00-06:00,o,#7570b3
1096,2023621288,TRAF - ACCIDENT - FATAL,2023-11-18 08:54:00.000001+00:00,N SPEER BLVD / N GROVE ST,False,-105.026852,39.761083,West Highland,0.0,0.0,...,2023-11-18,2023-11-18 Sat,"Sat Nov 18, 1:54 AM",2023,322,0101000020E6100000F8CAD5F2B7415AC09A4C2E296BE1...,0101000020AE0D00002970F07F2DE7474180A0B1FF32FA...,2023-11-18 01:54:00.000001-07:00,o,#7570b3
